# From detected synch cues, produce restamped raw Equivital and fixed intervals on march 6th, Wednesday
Note this interval didn't include any specific experiment intervals, so we are just generating the realigned files incase they end up being useful for different analyses later.

In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json 
from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate
from scipy.interpolate import interp1d

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex
%aimport al

In [3]:
def remove_tz_from_dataframe(df_in):
    df = df_in.copy()
    col_times = [ col for col in df.columns if any([isinstance(x, pd.Timestamp) for x in df[col]])]
    for col in col_times:
        df[col] = pd.to_datetime(df[col],format='ISO8601') 
        df[col] = df[col].dt.tz_localize(None) 
    return df

In [5]:
cues_df = pd.read_csv('./outfiles/'+'Cue_ACC_dev_align_20240306.csv',index_col = 0)
cues_df = cues_df.loc[cues_df['Cue1_s'].notna(),:].copy()
for c in cues_df.columns:
    if c.endswith('dt') or c.endswith('dTime'):
        if c.startswith('dur'):
            cues_df[c] = pd.to_timedelta(cues_df[c])
        else:
            cues_df[c] = pd.to_datetime(cues_df[c])
cues_df

,Cue1_devdTime,Cue1_s,Cue1_r,devID
DevName,,,,
VN101,2024-03-06 07:12:07.890625000,1.109375,0.566995,3420415
VN104,2024-03-06 07:12:08.398437500,0.601562,0.453916,3420456
VN105,2024-03-06 07:12:07.121093750,1.878906,0.450958,3420755
VN109,2024-03-06 07:12:07.328125000,1.671875,0.469074,5022010
VN110,2024-03-06 07:12:08.507812500,0.492188,0.469037,5022026
VN111,2024-03-06 07:12:07.046875000,1.953125,0.447598,5022027
VN206,2024-03-06 07:12:07.792968750,1.207031,0.516876,5022062
VN201,2024-03-06 07:12:08.562500000,0.437500,0.536402,5022050
VN211,2024-03-06 07:12:07.625000000,1.375000,0.556636,5022247


In [6]:
# {'cue1_20240309': Timestamp('2024-03-09 12:29:20.294512472'),
#  'cue2_20240309': Timestamp('2024-03-09 16:43:11.326464853')}
df_timings = pd.DataFrame(columns=['Event','Event_Type','SS_sTime','SS_dTime'])
Event_data = {'Event':'','Event_Type':'','SS_sTime':0,'SS_dTime':pd.to_timedelta('nan')}
# start of the synch spider audio 
sync_start = (pd.to_datetime('2024-03-06 07:12:08.000000000')).tz_localize(None)

In [8]:
# 19059.7765
i=0
event_dets = Event_data.copy()
event_dets['Event']='Sync'
event_dets['Event_Type']='Sync_Start'
event_dets['SS_sTime']= 0 # actually the alignment with synch spider recording
event_dets['SS_dTime']= pd.to_datetime('2024-03-06 07:12:08.000000000').tz_localize(None)
df_timings.loc[i,:]=event_dets


df_timings

,Event,Event_Type,SS_sTime,SS_dTime
0,Sync,Sync_Start,0,2024-03-06 07:12:08


In [16]:
# the problem recording is 'VN104'
# there is no end cue, so all using median_C
df_drifts=pd.read_csv('./outfiles/'+'dev_drift.csv', index_col=0)
df_drifts.loc['VN104','median_C']
df_drifts

,devID,mean_drift,median_drift,std_drift,mean_C,median_C,std_C,20240308_C,20240308_drift,20240309_C,20240309_drift,20240307_C,20240307_drift,20240305_C,20240305_drift
AU808,5022031,-0.245861,-0.245861,NaN,1.000068,1.000068,NaN,1.000068,-0.245861,NaN,NaN,NaN,NaN,NaN,NaN
AU807,5022058,-0.284143,-0.284143,NaN,1.000079,1.000079,NaN,1.000079,-0.284143,NaN,NaN,NaN,NaN,NaN,NaN
AU806,5022155,-0.251253,-0.251253,NaN,1.000070,1.000070,NaN,1.000070,-0.251253,NaN,NaN,NaN,NaN,NaN,NaN
AU805,5022002,-0.258802,-0.258802,NaN,1.000072,1.000072,NaN,1.000072,-0.258802,NaN,NaN,NaN,NaN,NaN,NaN
AU804,5022184,-0.280908,-0.280908,NaN,1.000078,1.000078,NaN,1.000078,-0.280908,NaN,NaN,NaN,NaN,NaN,NaN
AU801,5022179,-0.246401,-0.246401,NaN,1.000068,1.000068,NaN,1.000068,-0.246401,NaN,NaN,NaN,NaN,NaN,NaN
AU809,5022060,-0.281986,-0.281986,NaN,1.000078,1.000078,NaN,1.000078,-0.281986,NaN,NaN,NaN,NaN,NaN,NaN
AU810,5022098,-0.258262,-0.258262,NaN,1.000072,1.000072,NaN,1.000072,-0.258262,NaN,NaN,NaN,NaN,NaN,NaN
AU811,5022173,-0.269585,-0.269585,NaN,1.000075,1.000075,NaN,1.000075,-0.269585,NaN,NaN,NaN,NaN,NaN,NaN
AU813,5022244,-0.194199,-0.194199,0.044849,1.000054,1.000054,1.245942e-05,1.000063,-0.225912,1.000045,-0.162486,NaN,NaN,NaN,NaN


In [12]:
df_shifts=pd.read_csv('./outfiles/Cue_ACC_dev_align_20240306.csv', index_col=0)
df_shifts = df_shifts.loc[df_shifts['Cue1_s'].notna(),:].copy()

df_shifts['Cue1_devdTime'] = pd.to_datetime(df_shifts['Cue1_devdTime']).dt.tz_localize(None)
df_shifts

,Cue1_devdTime,Cue1_s,Cue1_r,devID
DevName,,,,
VN101,2024-03-06 07:12:07.890625000,1.109375,0.566995,3420415
VN104,2024-03-06 07:12:08.398437500,0.601562,0.453916,3420456
VN105,2024-03-06 07:12:07.121093750,1.878906,0.450958,3420755
VN109,2024-03-06 07:12:07.328125000,1.671875,0.469074,5022010
VN110,2024-03-06 07:12:08.507812500,0.492188,0.469037,5022026
VN111,2024-03-06 07:12:07.046875000,1.953125,0.447598,5022027
VN206,2024-03-06 07:12:07.792968750,1.207031,0.516876,5022062
VN201,2024-03-06 07:12:08.562500000,0.437500,0.536402,5022050
VN211,2024-03-06 07:12:07.625000000,1.375000,0.556636,5022247


In [15]:
data_dir='/Volumes/SeagateHub/BiC2024/20240306/Equivital/Extracted SEM Data/'

sep = '/'
projectName=''
df_datafiles = qex.qiosk_recordings(data_dir,projectName,sep)
# this set of files wasn't cleaned of the little files generated by accident
df_datafiles.set_index('DevName',inplace=True)
# df_datafiles = df_datafiles.loc[df_shifts.index].copy()
df_datafiles

,Signal,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,Duration,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
DevName,,,,,,,,,,,,,,,,,,,,,
VN101,DATA,3420415,240306,0,DATA-VN101-3420415-24030600.CSV,csv,108868,2024-03-06 07:01:28+00:00,2024-03-06 12:18:58+00:00,19050.0,...,[VN101],3420415.0,VN101,53.0,69,98.0,0.0,0.0,MovingSlowly,Upright
VN104,DATA,3420456,240306,101,DATA-VN104-3420456-24030600_0.CSV,csv,107663,2024-03-06 07:01:29+00:00,2024-03-06 12:17:59+00:00,18990.0,...,[VN104],3420456.0,VN104,53.0,83,98.0,0.0,0.0,Stationary,Upright
VN105,DATA,3420755,240306,0,DATA-VN105-3420755-24030600.CSV,csv,107418,2024-03-06 07:01:31+00:00,2024-03-06 12:19:03+00:00,19052.0,...,[VN105],3420755.0,VN105,53.0,0,98.0,0.0,0.0,Stationary,Upright
VN109,DATA,5022010,240306,0,DATA-VN109-5022010-24030600.CSV,csv,107678,2024-03-06 07:01:31+00:00,2024-03-06 12:19:01+00:00,19050.0,...,[VN109],5022010.0,VN109,53.0,81,98.0,0.0,0.0,Stationary,Upright
VN110,DATA,5022026,240306,0,DATA-VN110-5022026-24030600.CSV,csv,107918,2024-03-06 07:01:35+00:00,2024-03-06 12:19:05+00:00,19050.0,...,[VN110],5022026.0,VN110,53.0,79,98.0,0.0,0.0,Stationary,Upright
VN111,DATA,5022027,240306,0,DATA-VN111-5022027-24030600.CSV,csv,107913,2024-03-06 07:01:35+00:00,2024-03-06 12:18:20+00:00,19005.0,...,[VN111],5022027.0,VN111,53.0,80,95.0,0.0,0.0,Stationary,Upright
VN206,DATA,5022062,240306,0,DATA-VN206-5022062-24030600.CSV,csv,107062,2024-03-06 07:02:53+00:00,2024-03-06 12:17:53+00:00,18900.0,...,[VN206],5022062.0,VN206,53.0,66,99.0,0.0,0.0,Stationary,Upright
VN201,DATA,5022050,240306,0,DATA-VN201-5022050-24030600.CSV,csv,107890,2024-03-06 07:02:53+00:00,2024-03-06 12:20:08+00:00,19035.0,...,[VN201],5022050.0,VN201,53.0,75,97.0,0.0,0.0,Stationary,Upright
VN211,DATA,5022247,240306,0,DATA-VN211-5022247-24030600.CSV,csv,107380,2024-03-06 07:02:55+00:00,2024-03-06 12:18:40+00:00,18945.0,...,[VN211],5022247.0,VN211,53.0,77,97.0,0.0,0.0,Stationary,Upright


# Output full and for each event


In [17]:
df_datafiles

,Signal,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,Duration,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
DevName,,,,,,,,,,,,,,,,,,,,,
VN101,DATA,3420415,240306,0,DATA-VN101-3420415-24030600.CSV,csv,108868,2024-03-06 07:01:28+00:00,2024-03-06 12:18:58+00:00,19050.0,...,[VN101],3420415.0,VN101,53.0,69,98.0,0.0,0.0,MovingSlowly,Upright
VN104,DATA,3420456,240306,101,DATA-VN104-3420456-24030600_0.CSV,csv,107663,2024-03-06 07:01:29+00:00,2024-03-06 12:17:59+00:00,18990.0,...,[VN104],3420456.0,VN104,53.0,83,98.0,0.0,0.0,Stationary,Upright
VN105,DATA,3420755,240306,0,DATA-VN105-3420755-24030600.CSV,csv,107418,2024-03-06 07:01:31+00:00,2024-03-06 12:19:03+00:00,19052.0,...,[VN105],3420755.0,VN105,53.0,0,98.0,0.0,0.0,Stationary,Upright
VN109,DATA,5022010,240306,0,DATA-VN109-5022010-24030600.CSV,csv,107678,2024-03-06 07:01:31+00:00,2024-03-06 12:19:01+00:00,19050.0,...,[VN109],5022010.0,VN109,53.0,81,98.0,0.0,0.0,Stationary,Upright
VN110,DATA,5022026,240306,0,DATA-VN110-5022026-24030600.CSV,csv,107918,2024-03-06 07:01:35+00:00,2024-03-06 12:19:05+00:00,19050.0,...,[VN110],5022026.0,VN110,53.0,79,98.0,0.0,0.0,Stationary,Upright
VN111,DATA,5022027,240306,0,DATA-VN111-5022027-24030600.CSV,csv,107913,2024-03-06 07:01:35+00:00,2024-03-06 12:18:20+00:00,19005.0,...,[VN111],5022027.0,VN111,53.0,80,95.0,0.0,0.0,Stationary,Upright
VN206,DATA,5022062,240306,0,DATA-VN206-5022062-24030600.CSV,csv,107062,2024-03-06 07:02:53+00:00,2024-03-06 12:17:53+00:00,18900.0,...,[VN206],5022062.0,VN206,53.0,66,99.0,0.0,0.0,Stationary,Upright
VN201,DATA,5022050,240306,0,DATA-VN201-5022050-24030600.CSV,csv,107890,2024-03-06 07:02:53+00:00,2024-03-06 12:20:08+00:00,19035.0,...,[VN201],5022050.0,VN201,53.0,75,97.0,0.0,0.0,Stationary,Upright
VN211,DATA,5022247,240306,0,DATA-VN211-5022247-24030600.CSV,csv,107380,2024-03-06 07:02:55+00:00,2024-03-06 12:18:40+00:00,18945.0,...,[VN211],5022247.0,VN211,53.0,77,97.0,0.0,0.0,Stationary,Upright


In [20]:
CUTLOGS = pd.concat([df_datafiles,df_shifts],axis=1)#,CUTLOGS
CUTS= pd.Series()
# CUTS[pid] = []
sync1_dt = df_timings.loc[0,'SS_dTime']

newdata_dir='/Volumes/SeagateHub/BiC2024_Synced/Equivital/'

sig_map = {'DATAloc':'EQDATA','ACCloc':'EQBACC','RESPloc':'EQRESP','RRloc':'EQCIBI','ECGloc':'EQ2ECG'}
cut_ind = 0
toc = time.time()
rec_day = '20240306'
buff = pd.to_timedelta(15,unit='s') # 2 minute buffer on cutes for concert and stuff pd.to_timedelta(RecTimeSig['rec_sTime'] - ap['Cue0_rst'],unit='s')
segmentid = 'FULL'

for pid in CUTLOGS.index: # had to restart after VN212 as that is missing 
    tic = time.time()
    outfiles = []
    tic = time.time()
    # no second cue, so only using median drift 
    drift_C = df_drifts.loc[pid,'median_C']

    eqfiles = df_datafiles.loc[pid,['DATAloc','ACCloc','RESPloc','RRloc','ECGloc']]
    for f in eqfiles.index:
        SignalID = sig_map[f]
        V = pd.read_csv(eqfiles[f],skipinitialspace=True)
        dt_col = V.columns[0] # assume first column is time, specifically datetime
        sig_col = V.columns[1:]
        V[dt_col]=pd.to_datetime(V[dt_col]).dt.tz_localize(None) # .dt.tz_localize('Europe/Oslo').dt.tz_convert('UTC')
        V['rTime_s'] =np.round((V[dt_col]-df_shifts.loc[pid,'Cue1_devdTime']).dt.total_seconds()*drift_C,4)+df_timings.loc[0,'SS_sTime']
        V['rTime_dt'] = pd.to_timedelta(V['rTime_s']-df_timings.loc[0,'SS_sTime'],unit='s')+ sync1_dt
        RecTimeSig = pd.DataFrame()
        RecTimeSig['rec_sTime'] = np.round((V[dt_col]-df_shifts.loc[pid,'Cue1_devdTime']).dt.total_seconds()*drift_C,4)+df_timings.loc[0,'SS_sTime']
        RecTimeSig['rec_dTime'] = pd.to_timedelta(RecTimeSig['rec_sTime']-df_timings.loc[0,'SS_sTime'],unit='s')+ sync1_dt
        RecTimeSig['dev_dTime'] = V[dt_col]
        RecTimeSig.loc[:,sig_col] = V[sig_col]
        
        seg_filename = '_'.join([pid,rec_day,'FULL',SignalID+'.csv'])
        new_col_order=['rec_sTime']+list(sig_col) + ['rec_dTime','dev_dTime']
        RecTimeSig[new_col_order].to_csv(newdata_dir + 'FullDay/'+ rec_day+ '/'+ seg_filename, index = False)

        new_col_order=['c_sTime']+list(sig_col) + ['rec_dTime','dev_dTime']
        
        print(' '.join([pid,SignalID ,str(np.round(time.time() - tic,3)),str(np.round(time.time() - toc,3))]))
    CUTS[pid] = outfiles

# CUTS= pd.Series()
# CUTS[pid] = []
CUTLOGS['Output_files']=CUTS
CUTLOGS.to_csv('./outfiles/'+rec_day+'EQIVITALS_CUTLOGS.csv')
# CUTLOGS['Scaling_C'].hist()
# plt.show()
CUTLOGS

VN101 EQDATA 0.043 0.044
VN101 EQBACC 46.049 46.05
VN101 EQRESP 50.673 50.674
VN101 EQCIBI 50.866 50.867
VN101 EQ2ECG 99.607 99.608
VN104 EQDATA 0.106 99.715
VN104 EQBACC 46.636 146.245
VN104 EQRESP 51.231 150.84
VN104 EQCIBI 51.452 151.061
VN104 EQ2ECG 101.186 200.795
VN105 EQDATA 0.107 200.902
VN105 EQBACC 47.392 248.187
VN105 EQRESP 51.977 252.772
VN105 EQCIBI 52.174 252.97
VN105 EQ2ECG 102.338 303.133
VN109 EQDATA 0.111 303.245
VN109 EQBACC 45.927 349.062
VN109 EQRESP 50.69 353.825
VN109 EQCIBI 50.995 354.129
VN109 EQ2ECG 102.481 405.616
VN110 EQDATA 0.11 405.727
VN110 EQBACC 46.281 451.898
VN110 EQRESP 50.828 456.445
VN110 EQCIBI 51.141 456.758
VN110 EQ2ECG 101.271 506.888
VN111 EQDATA 0.117 507.005
VN111 EQBACC 45.814 552.703
VN111 EQRESP 50.13 557.018
VN111 EQCIBI 50.48 557.369
VN111 EQ2ECG 98.189 605.077
VN206 EQDATA 0.111 605.19
VN206 EQBACC 45.466 650.545
VN206 EQRESP 49.743 654.821
VN206 EQCIBI 49.994 655.073
VN206 EQ2ECG 97.211 702.29
VN201 EQDATA 0.099 702.39
VN201 EQBACC 

,Signal,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,Duration,...,HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION,Cue1_devdTime,Cue1_s,Cue1_r,devID,Output_files
DevName,,,,,,,,,,,,,,,,,,,,,
VN101,DATA,3420415,240306,0,DATA-VN101-3420415-24030600.CSV,csv,108868,2024-03-06 07:01:28+00:00,2024-03-06 12:18:58+00:00,19050.0,...,98.0,0.0,0.0,MovingSlowly,Upright,2024-03-06 07:12:07.890625000,1.109375,0.566995,3420415,[]
VN104,DATA,3420456,240306,101,DATA-VN104-3420456-24030600_0.CSV,csv,107663,2024-03-06 07:01:29+00:00,2024-03-06 12:17:59+00:00,18990.0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.398437500,0.601562,0.453916,3420456,[]
VN105,DATA,3420755,240306,0,DATA-VN105-3420755-24030600.CSV,csv,107418,2024-03-06 07:01:31+00:00,2024-03-06 12:19:03+00:00,19052.0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.121093750,1.878906,0.450958,3420755,[]
VN109,DATA,5022010,240306,0,DATA-VN109-5022010-24030600.CSV,csv,107678,2024-03-06 07:01:31+00:00,2024-03-06 12:19:01+00:00,19050.0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.328125000,1.671875,0.469074,5022010,[]
VN110,DATA,5022026,240306,0,DATA-VN110-5022026-24030600.CSV,csv,107918,2024-03-06 07:01:35+00:00,2024-03-06 12:19:05+00:00,19050.0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.507812500,0.492188,0.469037,5022026,[]
VN111,DATA,5022027,240306,0,DATA-VN111-5022027-24030600.CSV,csv,107913,2024-03-06 07:01:35+00:00,2024-03-06 12:18:20+00:00,19005.0,...,95.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.046875000,1.953125,0.447598,5022027,[]
VN206,DATA,5022062,240306,0,DATA-VN206-5022062-24030600.CSV,csv,107062,2024-03-06 07:02:53+00:00,2024-03-06 12:17:53+00:00,18900.0,...,99.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.792968750,1.207031,0.516876,5022062,[]
VN201,DATA,5022050,240306,0,DATA-VN201-5022050-24030600.CSV,csv,107890,2024-03-06 07:02:53+00:00,2024-03-06 12:20:08+00:00,19035.0,...,97.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.562500000,0.437500,0.536402,5022050,[]
VN211,DATA,5022247,240306,0,DATA-VN211-5022247-24030600.CSV,csv,107380,2024-03-06 07:02:55+00:00,2024-03-06 12:18:40+00:00,18945.0,...,97.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.625000000,1.375000,0.556636,5022247,[]
